# Hent historiske vejrdata
Download daglige historiske vejrdatadata i et angivet interval fra [Wunderground API](http://api.wunderground.com/weather/api/d/docs?d=data/history&MR=1)

http://frederiksberg-vejret.dk/

https://www.wunderground.com/personal-weather-station/dashboard?ID=IFREDERI180

In [18]:
import requests
import json
import time
from datetime import datetime, date
from dateutil.rrule import rrule, DAILY

In [59]:
class Weather:
    
    def __init__(self, api_key):
        self.api_key = api_key

    def get_weather(self, date, city):
        """
        Get historic weather data from Wunderground API.
        """
        url = f"http://api.wunderground.com/api/{self.api_key}/history_{date}/q/Denmark/{city}.json"
        response = requests.get(url)
        data = response.json()
        
        return data, date
    
    def save_data(self, path, data, file_name):
        """
        Save json to disk
        """
        with open(path + str(file_name) + '.json', 'w') as f:
            json.dump(data, f)
            
    def save_weather_period(self, path, city, start_date, end_date, sleep=6):
        """
        Saves weather data as json files to disk for each day in a historic period of time. 
        Sleem param is used to adjust request frequency (max 10 requests/min and 500 requests/day)
        """
        
        s_date = datetime.strptime(start_date, '%Y-%m-%d').date()
        e_date = datetime.strptime(end_date, '%Y-%m-%d').date()
        
        for dt in rrule(DAILY, dtstart=s_date, until=e_date):
            data, date = self.get_weather(dt.strftime("%Y%m%d"), city)
            self.save_data(path, data, date)
            time.sleep(sleep)

## Hent Data
Wunderground tillader max 10 kald/min / 500 kald/dag, derfor bruges time.sleep

In [60]:
api_key = '507f25f482fd4dc7'
weather = Weather(api_key)

In [61]:
path = 'data/weather/'
city = 'Copenhagen'
start_date = '2018-10-02'
end_date = '2018-10-03'

In [62]:
weather.save_weather_period(path, city, start_date, end_date)